## Rental owner clustering

Clusters rental license properties by owner name and owner address fields (ownname1, ownname2, ownadd1, ownadd2).  Writes the result to a temporary file.

In [2]:
!pip install leven
!pip install ngram

     |████████████████████████████████| 154 kB 7.3 MB/s 
  Created wheel for leven: filename=leven-1.0.4-cp37-cp37m-linux_x86_64.whl size=55506 sha256=d7df87b16b7f9e3d398db307f972d16a2ef1434c1a205f4be1bc7657a25a9343
  Stored in directory: /root/.cache/pip/wheels/27/d5/b1/b36dd23f85aa662f19b11ef6069d5dc66643dfc1831d9f2bf5
Successfully built leven
  Created wheel for ngram: filename=ngram-3.3.2-py3-none-any.whl size=24740 sha256=52ff7ff6c87b4fc6cae6d1cc8aae37ac2bfd53188a9526c90f1a688014ff29a0
  Stored in directory: /root/.cache/pip/wheels/24/09/ee/4d02ab106d48aa517b1295739e8f9c56ba159a3b61b43a64cf
Successfully built ngram


In [9]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Get cleaned rental registration and join it to SDAT, limited to Cambridge vs Dorchester County

In [25]:
rentals = pd.read_csv('/content/drive/My Drive/pita 2021/cleaned_rental_billing-2021.csv')
rentals.acctid = rentals.acctid.astype(str)

In [26]:
cambridge_apns = pd.read_csv('/content/drive/My Drive/SDAT/CAN-ref.csv')

In [42]:
sdat = pd.read_csv('drive/My Drive/pita 2021/SDAT-CAN-ref-202106.csv')

# make sure the acctid is in the right format, and limit the results to just Cambridge
sdat.acctid = sdat.acctid.apply(lambda x: str(x).strip())
sdat.city = sdat.city.fillna('')

#sdat = sdat.merge(cambridge_apns,on='acctid',how='outer',indicator=True)
#xtra_apns = sdat.query('_merge == "right_only"')
#xtra_categories = list(xtra_apns.CAN_OWNCLASS.unique())
#print("There are ",len(xtra_apns), "properties that don't match SDAT in the CAN ref.")
#print("None of these types impact the analysys?",xtra_categories)
#sdat = sdat.query('_merge == "both"').drop(columns='_merge').drop_duplicates(subset=['acctid'])
sdat[sdat.acctid == "1007229836"]
#sdat.query('~city.str.contains("HURLO")',engine='python')
#sdat[sdat.duplicated(subset=['acctid'])]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14,15,16,29,38,45,49,52,53,77,82,86,104,106,112,113,129,137) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,acctid,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,...,crtarcod,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address
11710,1007229836,DORC,478957.4,100526.9,2.401997e+10,2.401997e+11,81,N,SF,1212 ZACHARY DR,1212.0,NaN,ZACHARY,DR,NaN,NaN,P,CAMBRIDGE,21613.0,FOXTAIL CROSSING II LP,A MARYLAND LIMITED PARTNERSHIP,FOXTAIL CROSSING II LP,318 SIXTH ST STE 2,NaN,ANNAPOLIS,MD,21403.0,NaN,1212.0,NaN,ZACHARY,DR,CAMBRIDGE,21613.0,NaN,"IMPSTH 110R-4,200 SQFT.",S/S ZACHARY DR,FOXTAIL CROSSING,NaN,1092.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.56875257...,1216.0,40000.0,2013.0,1212.0,NaN,NaN,ZACHARY,DR,CAMBRIDGE,21613.0,1212 ZACHARY DR


In [43]:
import re
sdat.acctid = sdat.acctid.apply(lambda x: str(x).strip())
sdat.city = sdat.city.fillna('')

sdat.strtnum = sdat.strtnum.fillna(0).astype(int).astype(str)
sdat.strtdir = sdat.strtdir.fillna("")
sdat.strtnam = sdat.strtnam.fillna("")
sdat.strttyp = sdat.strttyp.fillna("")
sdat.strtunt = sdat.strttyp.fillna("")
sdat.premsnum = sdat.premsnum.fillna(0).astype(int).astype(str)
sdat.premsdir = sdat.premsdir.fillna("")
sdat.premsnam = sdat.premsnam.fillna("")
sdat.premstyp = sdat.premstyp.fillna("")
sdat.address = sdat.address.astype(str).fillna("").replace('nan','')

def get_best_sdat_address(id):
  x = sdat[sdat.acctid == id]
  x.strtnum = x.strtnum.fillna(0).astype(int).astype(str)
  x.strtdir = x.strtdir.fillna("")
  x.strtnam = x.strtnam.fillna("")
  x.strttyp = x.strttyp.fillna("")
  x.strtunt = x.strttyp.fillna("")
  x.premsnum = x.premsnum.fillna(0).astype(int).astype(str)
  x.premsdir = x.premsdir.fillna("")
  x.premsnam = x.premsnam.fillna("")
  x.premstyp = x.premstyp.fillna("")
  x.address = x.address.astype(str).fillna("").replace('nan','')
  strtnum = x.strtnum.item() if x.strtnum.item() != "0" else ""
  premsnum = x.premsnum.item() if x.premsnum.item() != "0" else ""
  address = x.address.values[0].strip()
  alt_address = ""

  # prefer unit number available
  strtdir = x.strtdir.item().strip()
  strtnam = x.strtnam.item().strip()
  strttyp = x.strttyp.item().strip()
  strtunt = x.strtunt.item().strip() if not x.strtunt.item().strip() == strttyp else ""
  alt_address = re.sub(r" +"," "," ".join([strtnum,str(strtdir),str(strtnam),str(strttyp),str(strtunt)])).strip().replace('nan','').replace(r' +',' ')
  # prefer sdat address of some sort, so see if 'premises' value is available
  if len(alt_address) == 0:
    strtdir = x.premsdir.item().strip()
    strtnam = x.premsnam.item().strip()
    strttyp = x.premstyp.item().strip()
    strtunt = ""
    alt_address = re.sub(r" +"," "," ".join([str(premsnum),str(strtdir),str(strtnam),str(strttyp),str(strtunt)])).strip().replace('nan','').replace(r' +',' ')

  # use the value from comcate if nothing else is available
  if (len(address) == 0 or address.isspace()) or (len(alt_address) or alt_address.isspace()):
    return x.legal2.item()#.fillna("").strip()
  else:
    return address if len(address) >= len(alt_address) else alt_address

get_best_sdat_address("1007229836")

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


'S/S ZACHARY DR'

In [44]:
sdat.address = sdat.apply(lambda x: get_best_sdat_address(x.acctid),axis=1)
print("done!")
print("WARNING:",len(rentals.drop(columns='address').merge(sdat,on='acctid',how='outer',indicator=True).query('_merge == "left_only"')),"didn't resolve!")
rentals = rentals.drop(columns='address').merge(sdat,on='acctid',how='inner')

done!


## Clustering rentals on owners requires joining with SDAT first

In [45]:
sdat.address = sdat.apply(lambda x: get_best_sdat_address(x.acctid),axis=1)
rentals.drop(columns='address').merge(sdat,on='acctid',how='outer',indicator=True).query('_merge == "left_only"')

,Unnamed: 0,License Id,License Type Id,Business Name,Customer Id,Issue Date,Effective Date,Expiration Date,State Id,Status,Property Location,Dist/Account No,Control Num,Phone,Phone Ext,Contact,Vin Id,Inspected By,Inspection Date,Seasonal,Insurance Co,Policy No,Insurance Exp Date,RENTAL,RR,acctid,jurscode_x,digxcord_x,digycord_x,ct2010_x,bg2010_x,geogcode_x,ooi_x,resityp_x,strtnum_x,strtdir_x,strtnam_x,strttyp_x,strtsfx_x,strtunt_x,...,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address,_merge


In [ ]:
!cp "drive/My Drive/pita 2021/text_cluster.py" .
from text_cluster import assign_clusters
from text_cluster import text_similarity
from text_cluster import cluster_strings

####Cluster the rental licenses
Assign clusters by owner name and owner address

In [ ]:
%%time
owner_names = cluster_strings(rentals.ownname1.array,'o_name',metric='jaccard',threshold=0.1)
owner_addresses = cluster_strings(rentals.ownadd1.astype(str).array,'o_address',metric='jaccard',threshold=0.1)

In [ ]:
def best_cluster(cnumbers):
  if len(cnumbers) > 1 and cnumbers.index[0] < 0:
    return cnumbers.index[1]
  else:
    return cnumbers.index[0]

rentals['owner_cluster'] = rentals.apply(lambda x: owner_names.get(x.ownname1,-1), axis=1)
rentals['owneradd_cluster'] = rentals.apply(lambda x: owner_addresses.get(x.ownadd1,-1), axis=1)
rentals[['acctid','ownname1','owner_cluster','ownadd1','owneradd_cluster']]

    
ro_df = pd.DataFrame(rentals.groupby(['acctid'])['owneradd_cluster'].agg(lambda x:best_cluster(x.value_counts())))#.to_dict()
roadd_c = rentals.groupby(['acctid'])['owner_cluster'].agg(lambda x:best_cluster(x.value_counts()))
ro_df = ro_df.merge(roadd_c,left_index=True,right_index=True)
ro_df

#### Save rental clusters

In [ ]:
ro_df.to_csv('drive/My Drive/pita 2021/rental_clusters.csv')